In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [4]:
FILENAME = 'tyme_20170101-20171231.csv'

def read_data(name):
    data = pd.read_csv(name)
    return data

In [ ]:
if __name__ == "__main__":
    data = read_data(FILENAME)